# First Class Functions

A programming language is said to have first-class functions if it treats functions as first-class citizens.(from wikipedia)

__First-Class Citizens__: A first-class citizen (also type, object, entity, or value) in a given programming language is an entity which supports all the operations generally available to other entities. These operations typically include __being passed as an argument, returned from a function, modified, and assigned to a variable__. (from wikipedia)

We should be able to treat functions just like any other object or a variable.

In [1]:
def square(x):
    return x*x

f = square(5)

print(square)
print(f)

<function square at 0x0000018BC99B4820>
25


In [5]:
def square(x):
    return x*x

f = square # 

print(square)
print(f)

print(f(5))

<function square at 0x0000018BC9A72D30>
<function square at 0x0000018BC9A72D30>
25


If a function accepts other functions as arguments or returns functions as their result i.e a __higher-order function__.

# Passing a function as an argument to another function

Ex: map function

In [7]:
def square(x):
    return x*x

def cube(x):
    return x*x*x

def my_map(func, arg_list): 
    result =[]
    for i in arg_list:
        result.append(func(i))
    return result
        
l=[1, 2, 3, 4, 5]
squares = my_map(square, l)
cubes = my_map(cube, l)
print(squares)
print(cubes)

[1, 4, 9, 16, 25]
[1, 8, 27, 64, 125]


# Return a function from another function

In [8]:
def logger(msg):
    
    def log_message():
        print('Log: ', msg)
        
    return log_message

log_hi = logger('Hi')
log_hi()

Log:  Hi


Why would returning a function from another function like this be useful?

  Ex:  Logging, Decorators

In [9]:
def html_tag(tag):
    
    def wrap_text(msg):
        print('<{0}>{1}</{0}>'.format(tag, msg))
    
    return wrap_text

print_h1 = html_tag('h1')
print_h1('This is heading')

print_p = html_tag('p')
print_p('This is Paragraph')

<h1>This is heading</h1>
<p>This is Paragraph</p>


# Closures

A closure is a record storing a function together with an environment. The environment is a mapping associating each free variable of the function (variables that are used locally, but defined in an enclosing scope) with the value or reference to which the name was bound when the closure was created. Unlike a plain function, a closure allows the function to access those captured variables through the closure's copies of their values or references, even when the function is invoked outside their scope. (from wikipedia)

In [14]:
def outer_func():
    message = 'Hi'
    
    def inner_func():
        print(message)
        
    return inner_func() #executed function

outer_func()

Hi


In [16]:
def outer_func():
    message = 'Hi'
    
    def inner_func():
        print(message)
        
    return inner_func #function that is waiting to be executed

my_func = outer_func()
print(my_func.__name__)
my_func()

inner_func
Hi


__Closure is an inner function that remembers and has access to variables in the local scope in which it was created even after the outer function has finished executing__.

In [17]:
def outer_func(msg):
    
    def inner_func():
        print(msg)
        
    return inner_func

hi_func = outer_func('Hi')
hello_func = outer_func('Hello')
hi_func()
hello_func()

Hi
Hello


In [18]:
import logging
logging.basicConfig(filename='example.log', level= logging.INFO)

def logger(func):
    def log_func(*args):
        logging.info(
            'Running "{}" function with arguments: {}'.format(func.__name__, args))
        print(func(*args))
    return log_func

def add(x, y):
    return x+y

def sub(x, y):
    return x-y

add_logger = logger(add)
sub_logger = logger(sub)

add_logger(3,3)
add_logger(4,5)

sub_logger(5,3)
sub_logger(20,10)

6
9
2
10


# Decorators

 A decorator is a function that takes another function as an argument and adds some kind of functionality and then returns another function. All of this without altering the source code of the original function that we passed in.

In [23]:
def decorator_function(original_func):
    def wrapper_function(): ##adds additional functionality to original_func
        print("Wrapper executed this before {}".format(original_func.__name__))
        print("****")
        return original_func()
    return wrapper_function

def display():
    print("Display function ran")
    
decorated_display = decorator_function(display)
decorated_display()

Wrapper executed this before display
****
Display function ran


 decorator_function returns a wrapper_function to be executed, when it is executed, it executes the original_func()

In [27]:
def decorator_function(original_func):
    def wrapper_function():
        print("Wrapper executed this before {}".format(original_func.__name__))
        print("****")
        return original_func()
    return wrapper_function

@decorator_function #1
def display(): #2
    print("Display function ran")
    
display()

## lines marked 1 and 2 are similar to display = decorator_function(display)

Wrapper executed this before display
****
Display function ran


In [30]:
def decorator_function(original_func):
    def wrapper_function(*args, **kwargs):
        print("Wrapper executed this before {}".format(original_func.__name__))
        print("****")
        return original_func(*args, **kwargs) #positional and keyword args
    return wrapper_function

@decorator_function 
def display():
    print("Display function ran")
    
@decorator_function
def display_info(name, age):
    print("Display_info ran with arguments {}, {}".format(name, age))
    
display()
display_info('John', 25)

Wrapper executed this before display
****
Display function ran
Wrapper executed this before display_info
****
Display_info ran with arguments John, 25


# Class as a decorator

In [31]:
def decorator_function(original_func):
    def wrapper_function(*args, **kwargs):
        print("Wrapper executed this before {}".format(original_func.__name__))
        print("****")
        return original_func(*args, **kwargs) #positional and keyword args
    return wrapper_function

class decorator_class:
    
    def __init__(self, original_function):
        self.original_function = original_function
        
    def __call__(self, *args, **kwargs): # acts as a wrapper function
        print("Call method executed this before {}".format(self.original_function.__name__))
        return self.original_function(*args, **kwargs)
@decorator_class
def display():
    print("Display function ran")
    
@decorator_class
def display_info(name, age):
    print("Display_info ran with arguments {}, {}".format(name, age))
    
display()
display_info('John', 25)

Call method executed this before display
Display function ran
Call method executed this before display_info
Display_info ran with arguments John, 25


# Practical examples

Common usecase for decorators in python is logging and timing how long each function runs .


In [39]:
import time

def my_logger(original_func):
    
    import logging
    logging.basicConfig(filename = '{}.log'.format(original_func.__name__), level= logging.INFO)
    
    def wrapper(*args, **kwargs):
        logging.info(
                    'ran with args {} and keyword args {}'.format(args, kwargs))
        return original_func(*args, **kwargs)
    
    return wrapper

def my_timer(original_func):
      
    def wrapper(*args, **kwargs):
        
        t1= time.time()
        result = original_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in {} secs".format(original_func.__name__, t2))
        
        return result
    
    return wrapper
    

@my_logger
def display_info(name, age):
    print("display_info ran with args {} {}".format(name, age))
    
display_info("Jane", 25)
display_info("Hank", 30)


@my_timer
def display():
    time.sleep(2)
    print("display ran")
    
display()

display_info ran with args Jane 25
display_info ran with args Hank 30
display ran
display ran in 2.001721143722534 secs


chain decorators together

In [40]:
@my_logger
@my_timer
def display_info(name, age):
    print("display_info ran with args {} {}".format(name, age))

display_info('ram', 25)

display_info ran with args ram 25
display_info ran in 0.0 secs


In [41]:
@my_timer
@my_logger
def display_info(name, age):
    print("display_info ran with args {} {}".format(name, age))

display_info('ram', 25)

display_info ran with args ram 25
wrapper ran in 0.0010013580322265625 secs


In [43]:
from functools import wraps
import time

def my_logger(original_func):
    
    import logging
    logging.basicConfig(filename = '{}.log'.format(original_func.__name__), level= logging.INFO)
    
    @wraps(original_func)
    def wrapper(*args, **kwargs):
        logging.info(
                    'ran with args {} and keyword args {}'.format(args, kwargs))
        return original_func(*args, **kwargs)
    
    return wrapper

def my_timer(original_func):
    
    @wraps(original_func)
    def wrapper(*args, **kwargs):
        
        t1= time.time()
        result = original_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in {} secs".format(original_func.__name__, t2))
        
        return result
    
    return wrapper


@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("display_info ran with args {} {}".format(name, age))
    
display_info("Jane", 25)
display_info("Hank", 30)




display_info ran with args Jane 25
display_info ran in 1.0010292530059814 secs
display_info ran with args Hank 30
display_info ran in 1.0020432472229004 secs
